# Import libraries

In [ ]:
import os
import torch
from src.data_loader.freihand_loader import F_DB
from torch.utils.data import DataLoader
from src.data_loader.utils import convert_2_5D_to_3D, get_root_depth, convert_to_2_5D
from src.visualization.visualize import plot_hand
from src.constants import MASTER_THESIS_DIR, FREIHAND_DATA, TRAINING_CONFIG_PATH
from src.utils import read_json
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
from tqdm.notebook import tqdm
from easydict import EasyDict as edict
from PIL import Image

In [ ]:
from src.data_loader.joints import Joints
joints = Joints()
config = edict(read_json(TRAINING_CONFIG_PATH))
# config.crop=False
# config.rotate =False
# config.resize=False

# Dataset

In [ ]:
f_db = F_DB(
    root_dir=os.path.join(FREIHAND_DATA, "training", "rgb"),
    labels_path=os.path.join(FREIHAND_DATA, "training_xyz.json"),
    camera_param_path=os.path.join(FREIHAND_DATA, "training_K.json"),
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            #             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    ),
    config=config
)

# Comparing original and recreated 3D

In [ ]:
# convert_2_5D_to_3D(f_db[0]['joints'], f_db[0]['scale'], f_db[0]['K'])
# print(f_db[0]['joints_3D'])

## Range of Error/absolute difference calculation

In [ ]:
error = torch.tensor(0)
error_min = torch.tensor(100)
high_error_index = []
for id in tqdm(range(len(f_db)//4)):
    joints25D = f_db[id]["joints"]
    scale = f_db[id]["scale"]
    K = f_db[id]["K"]
    true_joints_3D = f_db[id]["joints_3D"]
    error2  =torch.max(torch.abs((convert_2_5D_to_3D(joints25D, scale, K)-true_joints_3D)))
    error2min = torch.min(torch.abs((convert_2_5D_to_3D(joints25D, scale, K)-true_joints_3D)))
    if error2>error:
        error = error2
        print(f"error_max {error}")
    if error_min>error2min:
        error_min=error2min
        print(f"error_min {error_min}")

print(error_min, error)

# Augmentation exploration

## Visualize the cropping and rotation

### Inside the datareader

In [ ]:
@interact(
    id=widgets.IntSlider(min=0, max=1000, step=1, value=783),
)
def visualize_sample(id):
    sample = f_db[id]
    A = sample["joints"]
    s = sample["scale"]
    K = sample["K"]
    img = transforms.ToPILImage()(sample["image"])
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    plt.imshow(img)
    plot_hand(ax, A)
    plt.show()